## Version Description
In this version, we deploy a Hybrid PINN with adaptive Line Search in _full batch_ mode, to predict 5 correction factors in TZ6, viz. for the MTR and 4 CLRs

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# import required libraries
import pandas as pd
import numpy as np

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource,HoverTool
from bokeh.plotting import figure, show, output_file,output_notebook
from bokeh.io import export_svgs
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [3]:
# Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

### Data Acquisition

In [4]:
target_df = pd.read_csv('../data/TZ6_dataset.csv')
target_df.head()

,HoV,CAOLH_SumFlow,CAORH_SumFlow,LAOLH_SumFlow,LAORH_SumFlow,TZ6_Flow,MIXT,AMBT,AMBP,MIXP,...,R610_HS1,R611_HS1,R612_HS1,R613_HS1,R620_HS1,R620_HS2,R620_HS3,R621_HS1,R621_HS2,R621_HS3
0,A1,217.197005,222.443249,133.373448,136.693697,709.707399,293.150000,299.386667,101401.6000,2600.0000,...,131,136,120,120,75,38,113,75,38,113
1,A1,217.248929,223.698078,132.780168,135.684064,709.411240,293.150000,298.448667,101576.3000,2600.0000,...,131,136,120,120,75,38,113,75,38,113
2,A2,229.778639,226.831740,140.650844,143.718662,740.979885,298.101330,297.109024,102136.6035,2606.1928,...,131,136,114,120,75,38,113,75,38,113
3,A3,225.091032,227.788705,137.109226,142.122616,732.111579,294.390392,295.060027,103195.6642,2599.8998,...,131,136,120,120,75,38,113,75,38,113
4,A4,225.200989,222.557674,130.494552,135.055208,713.308423,293.150000,294.755833,102856.2000,2600.0000,...,145,153,130,130,75,38,113,75,38,113


In [5]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 21 columns):
HoV              35 non-null object
CAOLH_SumFlow    35 non-null float64
CAORH_SumFlow    35 non-null float64
LAOLH_SumFlow    35 non-null float64
LAORH_SumFlow    35 non-null float64
TZ6_Flow         35 non-null float64
MIXT             35 non-null float64
AMBT             35 non-null float64
AMBP             35 non-null float64
MIXP             35 non-null float64
R600_HD          35 non-null int64
R610_HS1         35 non-null int64
R611_HS1         35 non-null int64
R612_HS1         35 non-null int64
R613_HS1         35 non-null int64
R620_HS1         35 non-null int64
R620_HS2         35 non-null int64
R620_HS3         35 non-null int64
R621_HS1         35 non-null int64
R621_HS2         35 non-null int64
R621_HS3         35 non-null int64
dtypes: float64(9), int64(11), object(1)
memory usage: 5.8+ KB


In [6]:
# Define Restrictors and their parameters
MTR = 'R600_HD'
CLRS = ['R610_HS1','R611_HS1','R612_HS1','R613_HS1']

# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m
MHR_Duct_Areas = {'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461}
restrictor_duct_areas = {'R600_HD':0.03464,'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461 }

## Define Initial Correction factor values from CFD
TZ6_zmix_cf = 1 # Start with initial c_f =1 
MTR_cf = 1.1664876228437
MHR_cf = {'R610_HS1':0.740773983383846,'R611_HS1':0.740205245723713,'R612_HS1':0.83879652680329,'R613_HS1':0.81775586281569}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [7]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [8]:
_,target_df[MTR+'_Zeta1D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
# Gather all column names required for the input loss coefficients
zeta_col_names = [col for col in target_df.columns if 'Zeta1D' in col]
target_df[[MTR,'R600_HD_Zeta1D']].head()

,R600_HD,R600_HD_Zeta1D
0,148,4.949375
1,149,4.730821
2,152,4.127180
3,154,3.764504
4,148,4.949375


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [9]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [10]:
# Calculate Zeta for Multi-Hole Restrictors
for clr in CLRS:
    if (clr == 'R610_HS1') | (clr == 'R611_HS1'):
        hole_dia = 8
    else:
        hole_dia = 10
    MHR_nr_holes = target_df[clr].values
    _, target_df[clr+'_Zeta1D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])

In [11]:
# Gather all column names required for the input loss coefficients
from TZ6_FDDN_Lib import zeta_values
idx = target_df.columns.get_loc("R600_HD_Zeta1D")
target_df.insert(loc=(idx-1), column='TZ6_zmix_Zeta1D', value=float(zeta_values[0]))
zeta_col_names = [col for col in target_df.columns if 'Zeta1D' in col]

### Training data Preparation

In [12]:
from TZ6_5CF import NormbyMax, PINN

In [13]:
features2scale = ['R600_HD_Zeta1D','R610_HS1_Zeta1D', 'R611_HS1_Zeta1D','R612_HS1_Zeta1D','R613_HS1_Zeta1D','LAORH_SumFlow','LAOLH_SumFlow','CAORH_SumFlow','CAOLH_SumFlow','TZ6_Flow']
features_max, df_rescaled = NormbyMax(target_df,features2scale)
print('Max value of all features:',features_max)

Max value of all features: {'R600_HD_Zeta1D': 5.4148802645163, 'R610_HS1_Zeta1D': 10.02106896516872, 'R611_HS1_Zeta1D': 9.143664364318942, 'R612_HS1_Zeta1D': 10.11135599670757, 'R613_HS1_Zeta1D': 9.85771922403693, 'LAORH_SumFlow': 145.81587100000004, 'LAOLH_SumFlow': 144.888603, 'CAORH_SumFlow': 241.999479, 'CAOLH_SumFlow': 238.392638, 'TZ6_Flow': 764.0123719999998}


In [14]:
features_max_df = pd.DataFrame(data = features_max, index=[0])
print('Max value of all input features for normalization')
features_max_df

Max value of all input features for normalization


,R600_HD_Zeta1D,R610_HS1_Zeta1D,R611_HS1_Zeta1D,R612_HS1_Zeta1D,R613_HS1_Zeta1D,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow,TZ6_Flow
0,5.41488,10.021069,9.143664,10.111356,9.857719,145.815871,144.888603,241.999479,238.392638,764.012372


In [15]:
V_max_org = features_max_df[['TZ6_Flow','LAORH_SumFlow','LAOLH_SumFlow','CAORH_SumFlow','CAOLH_SumFlow']]
V_max_org

,TZ6_Flow,LAORH_SumFlow,LAOLH_SumFlow,CAORH_SumFlow,CAOLH_SumFlow
0,764.012372,145.815871,144.888603,241.999479,238.392638


In [16]:
# Manually selected Indices of train-val-test data split
train_data_idx = [16, 0, 4, 5, 6] # [16, 0, 4, 5, 6, 13, 28, 32, 14,  18, 1, 9, 12, 2, 20, 7, 3, 24, 27]
val_data_idx =  [17, 19, 21] # [17, 19, 21, 25, 33,  29, 8, 11,  22]
test_data_idx = [26, 34, 15, 26, 30, 10,  23,  31]

In [17]:
# Store indices and values of train data
input_features = ['R600_HD_Zeta1D','R610_HS1_Zeta1D', 'R611_HS1_Zeta1D','R612_HS1_Zeta1D','R613_HS1_Zeta1D']
train_data = df_rescaled[input_features].iloc[train_data_idx]
val_data = df_rescaled[input_features].iloc[val_data_idx]
test_data = df_rescaled[input_features].iloc[test_data_idx]
print("Number of training samples:", len(train_data), "\nNumber of validation samples:", len(val_data), "\nNumber of test samples:", len(test_data))

# Train data rparams
train_R600HD_series = target_df['R600_HD'].iloc[train_data_idx]
train_CLR610HN_series = target_df['R610_HS1'].iloc[train_data_idx]
train_CLR611HN_series = target_df['R611_HS1'].iloc[train_data_idx]
train_CLR612HN_series = target_df['R612_HS1'].iloc[train_data_idx]
train_CLR613HN_series = target_df['R613_HS1'].iloc[train_data_idx]
rparams_series_train = (train_R600HD_series,train_CLR610HN_series,train_CLR611HN_series,train_CLR612HN_series,train_CLR613HN_series)

# Validation data rparams
val_R600HD_series = target_df['R600_HD'].iloc[val_data_idx]
val_CLR610HN_series = target_df['R610_HS1'].iloc[val_data_idx]
val_CLR611HN_series = target_df['R611_HS1'].iloc[val_data_idx]
val_CLR612HN_series = target_df['R612_HS1'].iloc[val_data_idx]
val_CLR613HN_series = target_df['R613_HS1'].iloc[val_data_idx]
rparams_series_val = (val_R600HD_series, val_CLR610HN_series, val_CLR611HN_series, val_CLR612HN_series, val_CLR613HN_series)

# Test data rparams
test_R600HD_series = target_df['R600_HD'].iloc[test_data_idx]
test_CLR610HN_series = target_df['R610_HS1'].iloc[test_data_idx]
test_CLR611HN_series = target_df['R611_HS1'].iloc[test_data_idx]
test_CLR612HN_series = target_df['R612_HS1'].iloc[test_data_idx]
test_CLR613HN_series = target_df['R613_HS1'].iloc[test_data_idx]
rparams_series_test = (test_R600HD_series, test_CLR610HN_series, test_CLR611HN_series, test_CLR612HN_series, test_CLR613HN_series)

Number of training samples: 5 
Number of validation samples: 3 
Number of test samples: 8


### Predict $c_f$ with Physics Informed Neural Network

In [18]:
### Set np.random.seed() in the python script
### Set the hyperparameters here ###
nr_iterations = 25
learning_rates = [0, 0.001] ### IMPORTANT PARAMETER 
train_batch_size = len(train_data) 
val_batch_size = len(val_data)

N_i = train_data.shape[1] # No. of feature columns in the dataframe
hl_dim = [5] ### IMPORTANT PARAMETER
output_nodes = 5
network_dim = (N_i,hl_dim,output_nodes)


epsi_factor = 0.01
MTR_epsi = np.float(MTR_cf * epsi_factor)
CLR610_epsi = np.float(MHR_cf['R610_HS1'] * epsi_factor)
CLR611_epsi = np.float(MHR_cf['R611_HS1'] * epsi_factor)
CLR612_epsi = np.float(MHR_cf['R612_HS1'] * epsi_factor)
CLR613_epsi = np.float(MHR_cf['R613_HS1'] * epsi_factor)
epsi_tuple = (MTR_epsi, CLR610_epsi, CLR611_epsi, CLR612_epsi, CLR613_epsi)

#### Train data points sequentially - Single Data Point Training Mode

In [19]:
%%time
network = PINN(epsi_tuple, activation_function_choice = 'lrelu', error_loss_choice = 'full') #cf_only                                                                                                                                                                           
trained_model, MSE_flowloss_hist_train, error_loss_hist_train, cf_hist_train, MSE_flowloss_hist_val, error_loss_hist_val, cf_hist_val, flow_delta_val_mean = network.train_batch_LS(learning_rates, network_dim, input_features,train_data, rparams_series_train, train_data_idx, train_batch_size, restrictor_duct_areas, target_df, zeta_col_names, V_max_org, val_batch_size, val_data, rparams_series_val, val_data_idx, epochs=nr_iterations) 

Activation Function selected: linear
Loss Function & Delta-k Used: full

USING ENTIRE TRAIN DATASET in current TRAINING Batch: [16, 0, 4, 5, 6]
SEED value: 13
Nr of Hidden layers: 1 ;	Nr_Neurons in last hidden layer: 5
FIRST - INITIAL WEIGHTS & BIASES:
W1: [array([-0.319, 0.337, -0.0199, 0.202, 0.602]), array([0.238, 0.604, 0.385, 0.661, -0.468]), array([-0.353, -0.564, 0.252, -0.109, 0.409]), array([0.142, 0.0569, 0.962, 0.271, -0.012]), array([-0.44, 0.532, 0.426, -0.486, -0.0649])] <class 'numpy.ndarray'> Shape: (5, 5)
b1: [1.0, 1.0, 1.0, 1.0, 1.0] <class 'numpy.ndarray'> Shape: (5,)
W2: [array([0.106, -0.733, -0.124, 0.626, -0.722]), array([0.22, 0.846, -0.278, -0.203, 0.0972]), array([0.23, 0.178, -0.677, -0.342, 0.0453]), array([-0.142, 0.509, -0.144, -0.166, 0.827]), array([-0.0365, -0.438, -0.00107, -0.104, 0.318])] <class 'numpy.ndarray'> Shape: (5, 5)
b2: [1.0, 1.0, 1.0, 1.0, 1.0] <class 'numpy.ndarray'> Shape: (5,)

MTR-Cf: 1.7662665158826814 ; CLR-610-Cf: 2.819774509271218 

Predicted [MTR_cf, CLR610_cf, CLR611_cf, CLR612_cf, CLR613_cf]: (1.1270852123829314, 0.9677568331587755, 1.0171819735865215, 1.055223653066526, 0.9455276902568827)

EPOCH: 0 ;	Progress: 0.0% ;	MSE Flowloss (Train):  [21.94614303296215] ;	Error_loss (Train):  [0.3363280723698492] ;	MSE Flowloss (Val):  [40.08602204538178] ;	Error_loss (Val):  [0.48224763924449154]

MTR-Cf: 1.1269804387484095 ; CLR-610-Cf: 0.9982019582890554 ; CLR-611-Cf: 1.0112074831492945 ; CLR-612-Cf: 1.0247899978619193 ; CLR-613-Cf: 0.9987693417554281
Row ID: 16 	HoV: ['F1']

MTR-Cf: 1.1270852123829314 ; CLR-610-Cf: 0.9677568331587755 ; CLR-611-Cf: 1.0171819735865215 ; CLR-612-Cf: 1.055223653066526 ; CLR-613-Cf: 0.9455276902568827
Row ID: 0 	HoV: ['A1']

MTR-Cf: 1.0342952077939793 ; CLR-610-Cf: 0.7313061776500461 ; CLR-611-Cf: 1.280763889327833 ; CLR-612-Cf: 1.2521510530663496 ; CLR-613-Cf: 0.8121169852860387
Row ID: 4 	HoV: ['A4']

MTR-Cf: 1.1270852123829314 ; CLR-610-Cf: 0.9677568331587755 ; CLR-611-Cf: 1.017181973


MTR-Cf: 1.1010217550760686 ; CLR-610-Cf: 0.8944712194811255 ; CLR-611-Cf: 1.0469468924894736 ; CLR-612-Cf: 1.1066347147876814 ; CLR-613-Cf: 0.9125329044839332
Row ID: 0 	HoV: ['A1']

MTR-Cf: 1.0103107480630396 ; CLR-610-Cf: 0.6652100651394821 ; CLR-611-Cf: 1.307743894762384 ; CLR-612-Cf: 1.2988115778698208 ; CLR-613-Cf: 0.7827044667267591
Row ID: 4 	HoV: ['A4']

MTR-Cf: 1.1010217550760686 ; CLR-610-Cf: 0.8944712194811255 ; CLR-611-Cf: 1.0469468924894736 ; CLR-612-Cf: 1.1066347147876814 ; CLR-613-Cf: 0.9125329044839332
Row ID: 5 	HoV: ['A5']

MTR-Cf: 1.1010217550760686 ; CLR-610-Cf: 0.8944712194811255 ; CLR-611-Cf: 1.0469468924894736 ; CLR-612-Cf: 1.1066347147876814 ; CLR-613-Cf: 0.9125329044839332
Row ID: 6 	HoV: ['C1']
Flowrate_delta (FDDN - LTR): [array([0.434, -0.826, 2.06, 3.17, -3.98]), array([5.13, -2.02, 0.731, 1.52, 4.9]), array([16.9, 5.6, 3.74, 1.41, 6.16]), array([-7.31, -3.66, -1.84, -2.41, 0.598]), array([-0.404, -1.07, 1.84, 1.29, -2.47])] <class 'numpy.ndarray'> (5, 5)



MTR-Cf: 1.0322225674243464 ; CLR-610-Cf: 0.6599250173388358 ; CLR-611-Cf: 1.3112956806484175 ; CLR-612-Cf: 1.3006583619574204 ; CLR-613-Cf: 0.7831030861806749
Row ID: 4 	HoV: ['A4']

MTR-Cf: 1.1246748480627236 ; CLR-610-Cf: 0.8886753257370654 ; CLR-611-Cf: 1.050754796263913 ; CLR-612-Cf: 1.1086727845372568 ; CLR-613-Cf: 0.912854911134305
Row ID: 5 	HoV: ['A5']

MTR-Cf: 1.1246748480627236 ; CLR-610-Cf: 0.8886753257370654 ; CLR-611-Cf: 1.050754796263913 ; CLR-612-Cf: 1.1086727845372568 ; CLR-613-Cf: 0.912854911134305
Row ID: 6 	HoV: ['C1']
Flowrate_delta (FDDN - LTR): [array([-3.1, -1.38, 1.34, 1.99, -5.06]), array([1.71, -2.54, 0.0453, 0.378, 3.83]), array([13.4, 5.01, 3.07, 0.295, 5.07]), array([-10.7, -4.18, -2.52, -3.54, -0.458]), array([-3.79, -1.58, 1.15, 0.152, -3.52])] <class 'numpy.ndarray'> (5, 5)
f0: 0.7217935034256278 ;	lr0: 0 ;	MSE_FlowError: 19.582224249242323 ;	Error: 0.27855404852176324

MTR-Cf: 1.1219611473689417 ; CLR-610-Cf: 0.9095683773614645 ; CLR-611-Cf: 1.04685636


MTR-Cf: 1.1282267302007065 ; CLR-610-Cf: 0.864210157774194 ; CLR-611-Cf: 1.0579460694408533 ; CLR-612-Cf: 1.1074732956673086 ; CLR-613-Cf: 0.9082796510379678
Row ID: 0 	HoV: ['A1']

MTR-Cf: 1.0355521956574294 ; CLR-610-Cf: 0.6376869739888519 ; CLR-611-Cf: 1.317854988124973 ; CLR-612-Cf: 1.299363541926688 ; CLR-613-Cf: 0.7790928346841541
Row ID: 4 	HoV: ['A4']

MTR-Cf: 1.1282267302007065 ; CLR-610-Cf: 0.864210157774194 ; CLR-611-Cf: 1.0579460694408533 ; CLR-612-Cf: 1.1074732956673086 ; CLR-613-Cf: 0.9082796510379678
Row ID: 5 	HoV: ['A5']

MTR-Cf: 1.1282267302007065 ; CLR-610-Cf: 0.864210157774194 ; CLR-611-Cf: 1.0579460694408533 ; CLR-612-Cf: 1.1074732956673086 ; CLR-613-Cf: 0.9082796510379678
Row ID: 6 	HoV: ['C1']
Flowrate_delta (FDDN - LTR): [array([-3.36, -1.08, 1.07, 1.78, -5.14]), array([1.48, -2.25, -0.222, 0.184, 3.77]), array([13.2, 5.23, 2.85, 0.0992, 4.97]), array([-10.9, -3.89, -2.79, -3.74, -0.524]), array([-4.05, -1.29, 0.89, -0.0475, -3.6])] <class 'numpy.ndarray'> (5, 

...

TRAIN DATA - % Diff in Flowrates is LOW: [[0.534 0.406 0.361 0.0237 1.7]]
Flowrate_delta (FDDN - LTR): [array([-3.27, -0.406, 0.67, 1.84, -5.38]), array([1.72, -1.51, -0.624, 0.277, 3.58]), array([13, 5.73, 2.46, 0.0584, 4.73]), array([-10.7, -3.16, -3.17, -3.63, -0.705]), array([-3.79, -0.547, 0.476, 0.0524, -3.77])] <class 'numpy.ndarray'> (5, 5)
lr2 (Brent): 0.007472312474915389 ;	MSE_FlowError: 18.74313266859577 ;	Error: 0.27967118400750357

USING ENTIRE VALIDATION DATASET in current Batch: [17, 19, 21]

MTR-Cf: 1.1399268509023768 ; CLR-610-Cf: 0.8148753078431601 ; CLR-611-Cf: 1.033172813361951 ; CLR-612-Cf: 1.0456998441744032 ; CLR-613-Cf: 0.9352826436666086
Row ID: 17 	HoV: ['F2']
Predicted [MTR_cf, CLR610_cf, CLR611_cf, CLR612_cf, CLR613_cf]: (1.1399268509023768, 0.8148753078431601, 1.033172813361951, 1.0456998441744032, 0.9352826436666086)

MTR-Cf: 1.046374087169784 ; CLR-610-Cf: 0.567117126389181 ; CLR-611-Cf: 1.300106381421088 ; CLR-612-Cf: 1.269430998335404 ; CLR-613-Cf: 0.7

TRAIN DATA - % Diff in Flowrates is LOW: [[0.449 0.357 0.461 0.133 1.63]]
Flowrate_delta (FDDN - LTR): [array([-2.66, -0.324, 0.768, 2.1, -5.2]), array([2.35, -1.43, -0.507, 0.529, 3.76]), array([13.6, 5.81, 2.56, 0.287, 4.91]), array([-10.1, -3.09, -3.08, -3.4, -0.54]), array([-3.19, -0.481, 0.608, 0.294, -3.61])] <class 'numpy.ndarray'> (5, 5)
lr2 (Brent): 0.002242184136283812 ;	MSE_FlowError: 18.67569151143479 ;	Error: 0.27893169109011734

USING ENTIRE VALIDATION DATASET in current Batch: [17, 19, 21]

MTR-Cf: 1.1365268679476572 ; CLR-610-Cf: 0.8145351212980608 ; CLR-611-Cf: 1.0298786283087615 ; CLR-612-Cf: 1.0421820017544967 ; CLR-613-Cf: 0.9335832074604733
Row ID: 17 	HoV: ['F2']
Predicted [MTR_cf, CLR610_cf, CLR611_cf, CLR612_cf, CLR613_cf]: (1.1365268679476572, 0.8145351212980608, 1.0298786283087615, 1.0421820017544967, 0.9335832074604733)

MTR-Cf: 1.0432401857771512 ; CLR-610-Cf: 0.5667991517270224 ; CLR-611-Cf: 1.2970998550052752 ; CLR-612-Cf: 1.266212103133617 ; CLR-613-Cf: 0


MTR-Cf: 1.140087098921749 ; CLR-610-Cf: 0.7892888676835976 ; CLR-611-Cf: 1.0351343515540097 ; CLR-612-Cf: 1.072670530113701 ; CLR-613-Cf: 0.8824267843422515
Row ID: 6 	HoV: ['C1']
TRAIN DATA - % Diff in Flowrates is LOW: [[0.527 0.461 0.398 0.065 1.71]]
Flowrate_delta (FDDN - LTR): [array([-3.2, -0.472, 0.711, 1.94, -5.38]), array([1.78, -1.58, -0.582, 0.369, 3.58]), array([13, 5.68, 2.48, 0.132, 4.73]), array([-10.6, -3.23, -3.15, -3.55, -0.722]), array([-3.74, -0.622, 0.525, 0.144, -3.79])] <class 'numpy.ndarray'> (5, 5)
lr2 (Brent): 0.0014418784150260794 ;	MSE_FlowError: 18.741941292200632 ;	Error: 0.2788373228556883

USING ENTIRE VALIDATION DATASET in current Batch: [17, 19, 21]

MTR-Cf: 1.14008152747056 ; CLR-610-Cf: 0.8174729203350306 ; CLR-611-Cf: 1.0294324162969564 ; CLR-612-Cf: 1.0426728916574126 ; CLR-613-Cf: 0.9347036454307291
Row ID: 17 	HoV: ['F2']
Predicted [MTR_cf, CLR610_cf, CLR611_cf, CLR612_cf, CLR613_cf]: (1.14008152747056, 0.8174729203350306, 1.0294324162969564, 1.


MTR-Cf: 1.0433745341341467 ; CLR-610-Cf: 0.5710520337394969 ; CLR-611-Cf: 1.2941231363395413 ; CLR-612-Cf: 1.2649608610363736 ; CLR-613-Cf: 0.7544234823468942
Row ID: 4 	HoV: ['A4']

MTR-Cf: 1.1367280697032027 ; CLR-610-Cf: 0.7910633276092185 ; CLR-611-Cf: 1.0323257082054402 ; CLR-612-Cf: 1.0707921635730242 ; CLR-613-Cf: 0.8808932344965407
Row ID: 5 	HoV: ['A5']

MTR-Cf: 1.1367280697032027 ; CLR-610-Cf: 0.7910633276092185 ; CLR-611-Cf: 1.0323257082054402 ; CLR-612-Cf: 1.0707921635730242 ; CLR-613-Cf: 0.8808932344965407
Row ID: 6 	HoV: ['C1']
TRAIN DATA - % Diff in Flowrates is LOW: [[0.448 0.424 0.492 0.155 1.63]]
Flowrate_delta (FDDN - LTR): [array([-2.66, -0.423, 0.818, 2.15, -5.2]), array([2.33, -1.53, -0.465, 0.571, 3.76]), array([13.6, 5.73, 2.59, 0.32, 4.91]), array([-10.1, -3.18, -3.03, -3.35, -0.532]), array([-3.18, -0.572, 0.65, 0.344, -3.6])] <class 'numpy.ndarray'> (5, 5)
lr2 (Brent): 0.00234728622914722 ;	MSE_FlowError: 18.64974347919708 ;	Error: 0.2775165407336001

USING 


MTR-Cf: 1.1394077365871897 ; CLR-610-Cf: 0.79045563454234 ; CLR-611-Cf: 1.0328016845572354 ; CLR-612-Cf: 1.0715147155902327 ; CLR-613-Cf: 0.8811491221331109
Row ID: 0 	HoV: ['A1']

MTR-Cf: 1.0458581919107584 ; CLR-610-Cf: 0.5705013783558112 ; CLR-611-Cf: 1.294564671330411 ; CLR-612-Cf: 1.265623386049636 ; CLR-613-Cf: 0.754673036291093
Row ID: 4 	HoV: ['A4']

MTR-Cf: 1.1394077365871897 ; CLR-610-Cf: 0.79045563454234 ; CLR-611-Cf: 1.0328016845572354 ; CLR-612-Cf: 1.0715147155902327 ; CLR-613-Cf: 0.8811491221331109
Row ID: 5 	HoV: ['A5']

MTR-Cf: 1.1394077365871897 ; CLR-610-Cf: 0.79045563454234 ; CLR-611-Cf: 1.0328016845572354 ; CLR-612-Cf: 1.0715147155902327 ; CLR-613-Cf: 0.8811491221331109
Row ID: 6 	HoV: ['C1']
TRAIN DATA - % Diff in Flowrates is LOW: [[0.506 0.467 0.43 0.0875 1.68]]
Flowrate_delta (FDDN - LTR): [array([-3.08, -0.489, 0.752, 1.99, -5.33]), array([1.93, -1.59, -0.54, 0.428, 3.63]), array([13.1, 5.67, 2.51, 0.181, 4.78]), array([-10.5, -3.23, -3.11, -3.49, -0.656]), ar

In [20]:
print("MSE_flowloss_train_history:",MSE_flowloss_hist_train)

MSE_flowloss_train_history: [21.94614303296215, 21.943404521719568, 19.582224249242323, 19.50142433628174, 19.450867045868566, 19.310643155094386, 19.366219898201948, 19.194262818198265, 19.177008994059914, 19.04331640537787, 19.07005487829557, 19.024275689354333, 18.94532151439568, 18.904965957228736, 18.911274132035352, 18.864712285322643, 18.783387072251323, 18.91521980942753, 18.78169626055098, 18.72223429595384, 18.74313266859577, 18.67569151143479, 18.741941292200632, 18.64974347919708, 18.68101675469692]


In [21]:
print("MSE_flowloss_val_history:",MSE_flowloss_hist_val)

MSE_flowloss_val_history: [40.08602204538178, 51.926845110420636, 40.16270383412707, 43.06037358354254, 39.40904345584366, 41.691181833168436, 39.49448524391905, 41.561073465776985, 40.48162329776705, 41.425376022767786, 39.70087151291169, 42.79228214892612, 39.73307006217928, 40.68212070358104, 39.46499874681271, 39.878883989109504, 40.32391361380332, 38.3849968441077, 39.49692860904397, 39.8618382065596, 38.8657211974245, 40.29142699259168, 38.89542769922791, 40.22886875721224, 39.11407911262274]


In [22]:
print("Error_loss_train_history:",error_loss_hist_train)

Error_loss_train_history: [0.3363280723698492, 0.29284367910720943, 0.27855404852176324, 0.2761354261304314, 0.27629536301217, 0.274556639589706, 0.27535843275911315, 0.2742946931826692, 0.2743001824713306, 0.27553269470266484, 0.2755615479992647, 0.2769374715945092, 0.27654923313946006, 0.27613154335805107, 0.27639425781917865, 0.2758008691784385, 0.27675744916765743, 0.27789937166101336, 0.27659520493238987, 0.27639605742128864, 0.27967118400750357, 0.27893169109011734, 0.2788373228556883, 0.2775165407336001, 0.2780076054012545]


In [23]:
print("Error_loss_val_history:",error_loss_hist_val)

Error_loss_val_history: [0.48224763924449154, 0.5784599560652343, 0.4907278749381928, 0.518078802396472, 0.4905993570100598, 0.5108315987382127, 0.495929242406935, 0.514062481351716, 0.5055001652382355, 0.519368147274506, 0.5045250176553807, 0.5335256433090242, 0.5090862255438637, 0.5152987968385041, 0.50651036517741, 0.5087446492890145, 0.5118361401198808, 0.49503539669012275, 0.5038392843677241, 0.5071317099453213, 0.5086005072965272, 0.5187639824652627, 0.5059808957156183, 0.5145916764994082, 0.506123659049167]


In [24]:
cf_df_train = pd.DataFrame(data=cf_hist_train)
# cf_df_train.to_csv('data_output/NN_5CFs_Output_LS_fullBatch.csv',index=False)
cf_df_train

,HoV,MTR_cf_hat,CLR610_cf_hat,CLR611_cf_hat,CLR612_cf_hat,CLR613_cf_hat,Epoch
0,F1,1.123170,0.876145,1.057850,1.075615,0.960083,24
1,A1,1.123352,0.847288,1.063260,1.105349,0.907481,24
2,A4,1.031075,0.622295,1.322684,1.297281,0.778461,24
3,A5,1.123352,0.847288,1.063260,1.105349,0.907481,24
4,C1,1.123352,0.847288,1.063260,1.105349,0.907481,24


In [25]:
cf_df_val = pd.DataFrame(data=cf_hist_val)
cf_df_val

,HoV,MTR_cf_hat_val,CLR610_cf_hat_val,CLR611_cf_hat_val,CLR612_cf_hat_val,CLR613_cf_hat_val,Epoch
0,J1,1.124675,0.888675,1.050755,1.108673,0.912855,2
1,J1,1.118804,0.869267,1.054554,1.107561,0.907635,3
2,J1,1.128022,0.864320,1.057872,1.107475,0.908266,4
3,J1,1.122414,0.858329,1.059885,1.106437,0.908567,5
4,J1,1.128441,0.852056,1.062594,1.106560,0.908982,6
5,J1,1.123352,0.847288,1.063260,1.105349,0.907481,7
6,J1,1.126169,0.845515,1.063958,1.104392,0.907591,8
7,J1,1.126751,0.824784,1.060322,1.096984,0.900517,9
8,J1,1.130875,0.824937,1.060495,1.095879,0.900987,10
9,J1,1.125136,0.812267,1.057713,1.091313,0.895527,11


### Plot Training Losses

In [26]:
hover = [('Epoch', '@x'), ('Value','@y')]

**MSE FlowError Loss**

In [27]:
p1 = figure(title = 'Full batch MSE FlowLoss performance',width=1600, tooltips=hover)
p1.line(list(range(1, len(MSE_flowloss_hist_train)+1)), MSE_flowloss_hist_train, line_width=2, color='blue', alpha=0.8, legend = 'Train Data')
p1.line(list(range(1, len(MSE_flowloss_hist_val)+1)), MSE_flowloss_hist_val, line_width=2, color='green', alpha=0.8, legend = 'Validation Data')
p1.yaxis.axis_label = "MSE (FDDN_V, LTR_V)"
p1.xaxis.axis_label = "Epochs"
p1.legend.location = "top_right"
p1.legend.click_policy="hide"
show(p1)

**Error Loss**

In [28]:
p2 = figure(title = 'FullBatch Train and Validation Loss Results', width=1250, tooltips=hover)
p2.line(list(range(1,len(error_loss_hist_train)+1)), error_loss_hist_train, line_width= 2, color='green', alpha=0.8, legend='Train Loss')  
p2.circle(list(range(1,len(error_loss_hist_train)+1)), error_loss_hist_train, line_width= 5, color= 'green', alpha=0.75, size = 4,  legend='Train Loss')
p2.line(list(range(1,len(error_loss_hist_val)+1)), error_loss_hist_val, line_width= 2, line_dash = 'dashed', color='blue', alpha=0.8, legend='Validation Loss')  
p2.circle(list(range(1,len(error_loss_hist_val)+1)), error_loss_hist_val, line_width= 5, color= 'blue', alpha=0.45, size = 4, legend='Validation Loss')
p2.yaxis.axis_label = "Error Loss"
p2.xaxis.axis_label = "Epochs"
p2.legend.click_policy="hide"
p2.xaxis.axis_label_text_font_size = '12pt'
p2.yaxis.axis_label_text_font_size = '12pt'
p2.xaxis.major_label_text_font_size= '11pt'
p2.yaxis.major_label_text_font_size= '11pt'
p2.legend.label_text_font_size = '12pt'
p2.xaxis.major_label_standoff = 15
p2.output_backend = 'svg'
# export_svgs(p2, filename="plots/TZ6_5CF_fullbatch_Train_Val_Loss_Results.svg")
show(p2)

In [29]:
p3 = figure(title = 'Flow_delta_Mean Results', width=1250, tooltips=hover)
x = list(range(1,len(flow_delta_val_mean)+1))
p3.line(x, flow_delta_val_mean, line_width=2, color='tomato', alpha=0.8)
p3.circle(x, flow_delta_val_mean, line_width=5, color= 'tomato', alpha=0.9, size = 2)
p3.yaxis.axis_label = "Flow_delta_Mean"
p3.xaxis.axis_label = "Epochs"
p3.legend.click_policy="hide"
p3.xaxis.axis_label_text_font_size = '12pt'
p3.yaxis.axis_label_text_font_size = '12pt'
p3.xaxis.major_label_text_font_size= '11pt'
p3.yaxis.major_label_text_font_size= '11pt'
p3.legend.label_text_font_size = '12pt'
p3.xaxis.major_label_standoff = 15
p3.output_backend = 'svg'
# export_svgs(p3, filename="plots/TZ6_5CF_fullbatch_FlowDeltaMeanResults.svg")
show(p3)

### Make Predictions on Test Data Points

In [30]:
test_R600HD_series = final_df['R600_HD'].iloc[test_data_idx]
test_CLR610HN_series = final_df['R610_HS1'].iloc[test_data_idx]
test_CLR611HN_series = final_df['R611_HS1'].iloc[test_data_idx]
test_CLR612HN_series = final_df['R612_HS1'].iloc[test_data_idx]
test_CLR613HN_series = final_df['R613_HS1'].iloc[test_data_idx]
rparams_series_test = (test_R600HD_series, test_CLR610HN_series, test_CLR611HN_series, test_CLR612HN_series, test_CLR613HN_series)

NameError: name 'final_df' is not defined

In [ ]:
test_data_HoVs = final_df['HoV'].iloc[test_data_idx].values
V_true_LTR_testdata, V_hat_FDDN_testdata, delta_V_testdata = [], [], []
MTR_cfs_test_arr, CLR610_cfs_test_arr, CLR611_cfs_test_arr, CLR612_cfs_test_arr, CLR613_cfs_test_arr = [], [], [], [], []

for row_id, MTR_dia, CLR610_hn, CLR611_hn, CLR612_hn, CLR613_hn in zip(test_data_idx, test_R600HD_series, test_CLR610HN_series, test_CLR611HN_series, test_CLR612HN_series, test_CLR613HN_series):
    rparam = (MTR_dia, CLR610_hn, CLR611_hn, CLR612_hn, CLR613_hn)
    cfs_test_tuple, V_LTR, V_hat_FDDN, delta_V = network.predict(trained_model, test_data.loc[row_id].values, row_id, rparam, restrictor_duct_areas, zeta_col_names, target_df)  
    MTR_cf_test, CLR610_cf_test, CLR611_cf_test, CLR612_cf_test, CLR613_cf_test = cfs_test_tuple
    V_true_LTR_testdata.append(V_LTR)
    V_hat_FDDN_testdata.append(V_hat_FDDN)
    delta_V_testdata.append(delta_V)
    MTR_cfs_test_arr.append(MTR_cf_test)
    CLR610_cfs_test_arr.append(CLR610_cf_test)
    CLR611_cfs_test_arr.append(CLR611_cf_test)
    CLR612_cfs_test_arr.append(CLR612_cf_test)
    CLR613_cfs_test_arr.append(CLR613_cf_test)

    
testdata_cf_predictions_df = pd.DataFrame(data = {'HoV':test_data_HoVs, 'MTR_cf_hat':MTR_cfs_test_arr, 'CLR610_cf_hat':CLR610_cfs_test_arr, 'CLR611_cf_hat':CLR611_cfs_test_arr, 'CLR612_cf_hat':CLR612_cfs_test_arr, 'CLR613_cf_hat':CLR613_cfs_test_arr})

In [ ]:
delta_V_df = pd.concat(delta_V_testdata,ignore_index=True)
V_true_LTR_test_df = pd.concat(V_true_LTR_testdata,ignore_index=True)
V_hat_FDDN_test_df = pd.concat(V_hat_FDDN_testdata,ignore_index=True)

In [ ]:
print("LTR_Test_Data_Predictions",'\t\t\t\t\t\t\tFDDN_Test_data_predictions')
display_side_by_side(V_true_LTR_test_df, V_hat_FDDN_test_df)

In [ ]:
delta_V_df

In [ ]:
testdata_cf_predictions_df